In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import torch
import re

In [ ]:
#installing pretrained model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
tokens = tokenizer.encode('I hated this, absolutely the worst',return_tensors='pt')

In [ ]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
          102])

In [ ]:
tokenizer.decode(token[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [ ]:
result = model(tokens)

In [ ]:
#values represent the probability of that class to be the sentiment 
result


SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
#sentiment rating
int(torch.argmax(result.logits))+1

1

In [ ]:
tokens = tokenizer.encode('This is amazing, great!!',return_tensors='pt')

result = model(tokens)
#sentiment rating
int(torch.argmax(result.logits))+1

5

In [ ]:
tokens = tokenizer.encode('It was good but could have been better',return_tensors='pt')

result = model(tokens)
#sentiment rating
int(torch.argmax(result.logits))+1

3

##Collecting Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
results[0].text

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df.head()

,review
0,Great coffee and vibe. That's all you need. C...
1,Great coffee and vibe. That's all you need. C...
2,Great food amazing coffee and tea. Short walk ...
3,Great staff and food. Must try is the pan fri...
4,Ricotta hot cakes! These were so yummy. I ate ...


In [ ]:
df['review'].iloc[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1


In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512])) #limited to 512 tokens

In [ ]:
df

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,Great food amazing coffee and tea. Short walk ...,5
3,Great staff and food. Must try is the pan fri...,5
4,Ricotta hot cakes! These were so yummy. I ate ...,5
5,I came to Social brew cafe for brunch while ex...,5
6,We came for brunch twice in our week-long visi...,4
7,It was ok. The coffee wasn't the best but it w...,3
8,I went here a little while ago- a beautiful mo...,2
9,This place is a gem. The ambiance is to die fo...,3
